# Assumptions

1. Only account owners are important, account users play little part in my analysis
2. Date of Analysis assumed to be December 31, 1999. Right before the end of the world

# Imports

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

In [2]:
loan_df = pd.read_csv("data/loan_info.csv")
account_df = pd.read_csv('data/account_info.csv')
payment_df = pd.read_csv('data/payment_info.csv')

# Altering the Loans table

In [3]:
pd.set_option("display.max_columns", None)

In [5]:
loan_df.head()

,account_id,owner_client_id,owner_disp_id,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,loan_id,loan_date,loan_amt,total_paid,loan_duration_mos,total_payments,monthly_payments,loan_status,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,2,2,2,1945-02-04,POPLATEK MESICNE,NaN,NaN,1993-02-26,4959,1994-01-05,80952,80952.0,24,24,3373.0,A,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,19,25,25,1,1939-04-23,POPLATEK MESICNE,NaN,NaN,1995-04-07,4961,1996-04-29,30276,20184.0,12,8,2523.0,B,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,25,31,31,1,1962-02-09,POPLATEK MESICNE,NaN,NaN,1996-07-28,4962,1997-12-08,30276,30276.0,12,12,2523.0,A,68,Frydek - Mistek,north Moravia,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,37,45,45,1,1952-08-26,POPLATEK MESICNE,NaN,NaN,1997-08-18,4967,1998-10-14,318480,5308.0,60,1,5308.0,D,20,Strakonice,south Bohemia,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,38,46,46,1,1940-01-30,POPLATEK TYDNE,NaN,NaN,1997-08-08,4968,1998-04-19,110736,18456.0,48,8,2307.0,C,19,Prachatice,south Bohemia,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099


In [7]:
loan_df.dtypes

account_id             int64
owner_client_id        int64
owner_disp_id          int64
num_acct_users         int64
owner_birth_date      object
statement_freq        object
card_type             object
card_issue_date       object
acct_start_date       object
loan_id                int64
loan_date             object
loan_amt               int64
total_paid           float64
loan_duration_mos      int64
total_payments         int64
monthly_payments     float64
loan_status           object
district_id            int64
A2                    object
A3                    object
A4                     int64
A5                     int64
A6                     int64
A7                     int64
A8                     int64
A9                     int64
A10                  float64
A11                    int64
A12                  float64
A13                  float64
A14                    int64
A15                  float64
A16                    int64
dtype: object

In [8]:
# These columns will not be useful in the analysis
loan_df.drop(columns=['owner_disp_id', 'loan_id', 'owner_client_id'], inplace=True)

In [10]:
# Convert our datetime columns to actual datetimes
loan_df['owner_birth_date'] = pd.to_datetime(loan_df['owner_birth_date'], format='%Y-%m-%d')
loan_df['acct_start_date'] = pd.to_datetime(loan_df['acct_start_date'], format='%Y-%m-%d')
loan_df['card_issue_date'] = pd.to_datetime(loan_df['card_issue_date'], format='%Y-%m-%d')
loan_df['loan_date'] = pd.to_datetime(loan_df['loan_date'], format='%Y-%m-%d')

In [11]:
loan_df.head()

,account_id,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,loan_date,loan_amt,total_paid,loan_duration_mos,total_payments,monthly_payments,loan_status,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,2,1945-02-04,POPLATEK MESICNE,NaN,NaT,1993-02-26,1994-01-05,80952,80952.0,24,24,3373.0,A,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,19,1,1939-04-23,POPLATEK MESICNE,NaN,NaT,1995-04-07,1996-04-29,30276,20184.0,12,8,2523.0,B,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,25,1,1962-02-09,POPLATEK MESICNE,NaN,NaT,1996-07-28,1997-12-08,30276,30276.0,12,12,2523.0,A,68,Frydek - Mistek,north Moravia,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,37,1,1952-08-26,POPLATEK MESICNE,NaN,NaT,1997-08-18,1998-10-14,318480,5308.0,60,1,5308.0,D,20,Strakonice,south Bohemia,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,38,1,1940-01-30,POPLATEK TYDNE,NaN,NaT,1997-08-08,1998-04-19,110736,18456.0,48,8,2307.0,C,19,Prachatice,south Bohemia,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099


In [58]:
# A datetime object is not useful for analysis, so let's make the birthdates an age in years
loan_df['owner_age'] = ((datetime.datetime(1999,12,31)-loan_df['owner_birth_date']).dt.days/365.25).astype(int)

# Next we will want to know how old the loan is, but let's do so in months, since that's the payment plan
# Want to be more careful here - some loans are finished, so maybe just use the present loan_duration column
# loan_df['loan_length']

# Same for credit card, make it in months, since credit cards are billed monthly
loan_df['mos_with_card'] = np.floor((datetime.datetime(1999,12,31)-loan_df['card_issue_date']).dt.days/12)

# We will do months for Account length as well, since again, account fees are billed monthly
loan_df['account_len_mos'] = ((datetime.datetime(1999,12,31)-loan_df['acct_start_date']).dt.days/12).astype(int)

In [59]:
loan_df.head()

,account_id,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,loan_date,loan_amt,total_paid,loan_duration_mos,total_payments,monthly_payments,loan_status,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,owner_age,mos_with_card,account_length,account_len_mos
0,2,2,1945-02-04,POPLATEK MESICNE,NaN,NaT,1993-02-26,1994-01-05,80952,80952.0,24,24,3373.0,A,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,54,NaN,208,208
1,19,1,1939-04-23,POPLATEK MESICNE,NaN,NaT,1995-04-07,1996-04-29,30276,20184.0,12,8,2523.0,B,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354,60,NaN,144,144
2,25,1,1962-02-09,POPLATEK MESICNE,NaN,NaT,1996-07-28,1997-12-08,30276,30276.0,12,12,2523.0,A,68,Frydek - Mistek,north Moravia,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887,37,NaN,104,104
3,37,1,1952-08-26,POPLATEK MESICNE,NaN,NaT,1997-08-18,1998-10-14,318480,5308.0,60,1,5308.0,D,20,Strakonice,south Bohemia,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542,47,NaN,72,72
4,38,1,1940-01-30,POPLATEK TYDNE,NaN,NaT,1997-08-08,1998-04-19,110736,18456.0,48,8,2307.0,C,19,Prachatice,south Bohemia,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099,59,NaN,72,72
